# Research Experiments on the effect of Selection on Generalization for symbolic Regression in GP

* Masterseminar: SoSe 2022
* JGU Mainz
* FB 03 Recht-und Wirtschaftswissenschaften
* Lehrstuhl für Wirtschaftsinformatik und BWL

## Dependencies

In [1]:
import numpy as np
import pandas as pd
import operator
import os
import math
from copy import deepcopy
from deap import gp, tools, creator, base, algorithms
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Callable
from random import randint
from sys import stderr
from datetime import datetime

## Energy efficiency Data Set

Source: https://archive.ics.uci.edu/ml/datasets/energy+efficiency

In [2]:
if not os.path.exists("./data/ENB2012_data.xlsx"):
    os.system("wget https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx -P ./data")

In [3]:
def get_datasets() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Read .xlsx dataset at <D_PATH> and return two randomly split DFs for training/testing"""

    D_PATH = "data/ENB2012_data.xlsx"
    TRAINING_D_SPLITSIZE = 0.5

    df = pd.read_excel(D_PATH)

    return train_test_split(df, train_size=TRAINING_D_SPLITSIZE, test_size=(1-TRAINING_D_SPLITSIZE))    


TRAINING_SET, TESTING_SET = get_datasets()

In [4]:
TRAINING_SET.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
694,0.76,661.5,416.5,122.5,7.0,4,0.4,4,40.60,39.85
110,0.82,612.5,318.5,147.0,7.0,4,0.1,2,23.67,24.80
641,0.79,637.0,343.0,147.0,7.0,3,0.4,3,41.30,44.18
166,0.76,661.5,416.5,122.5,7.0,4,0.1,3,32.33,34.48
758,0.66,759.5,318.5,220.5,3.5,4,0.4,5,14.92,17.55


In [5]:
TRAINING_SET.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
count,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000
mean,0.775208,662.329427,317.734375,172.297526,5.395833,3.528646,0.235286,2.880208,22.958724,25.275521
std,0.107655,87.744646,44.148039,45.798413,1.746188,1.109754,0.132552,1.538445,9.860913,9.387178
min,0.620000,514.500000,245.000000,110.250000,3.500000,2.000000,0.000000,0.000000,6.010000,10.900000
25%,0.690000,588.000000,294.000000,122.500000,3.500000,3.000000,0.100000,2.000000,14.110000,16.152500
50%,0.760000,661.500000,318.500000,147.000000,7.000000,4.000000,0.250000,3.000000,23.900000,25.655000
75%,0.860000,735.000000,343.000000,220.500000,7.000000,5.000000,0.400000,4.000000,32.055000,33.370000
max,0.980000,808.500000,416.500000,220.500000,7.000000,5.000000,0.400000,5.000000,42.770000,46.440000


## Data Visualization

In [6]:
from matplotlib import pyplot as plt
import networkx as nx
import pygraphviz as pgv

%matplotlib inline

def plot_exprTree(expr_tree, title:str) -> None:
    """plots an expression tree"""
    nodes, edges, labels = gp.graph(expr_tree)

    g = nx.Graph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    
    pos = nx.drawing.nx_agraph.graphviz_layout(g, prog="dot")

    nx.draw_networkx_nodes(g, pos)
    nx.draw_networkx_edges(g, pos)
    nx.draw_networkx_labels(g, pos, labels)

    plt.title(title)
    plt.show()

## Implementing protected functions for GP

Source:
J.  Koza,  Genetic Programming: On the Programming of Computers by Means of Natural Selection (MIT Press, Cambridge, 1992)

In [7]:
def pdiv(lhs: float, rhs: float) -> float:
    """
    Koza Style implementation of division
    [@Koza2005]
    """
    if rhs == 0:
        return 1
    return lhs / rhs

def plog(x: float) -> float:
    """
    Koza Style implementation of natural logarithm
    [@Koza2005]
    """
    if x == 0:
        return 0
    return math.log(abs(x))
    

def psqrt(x: float) -> float:
    """
    Koza Style implementation of square root
    [@Koza2005]
    """
    return math.sqrt(abs(x))


def ppow(base: float, power: float) -> float:
    """
    Adjusted Implementation of power operator
    [@fsets_generalisation]
    """
    if (base != 0) or (base == power == 0):
        return abs(base) ** power
    return 0

## GP System Setup

In [8]:
NUM_GENERATIONS = 5

### Primitive set

In [9]:
uvs = {
    "ARG0" : "X1",
    "ARG1" : "X2",
    "ARG2" : "X3",
    "ARG3" : "X4",
    "ARG4" : "X5",
    "ARG5" : "X6",
    "ARG6" : "X7",
    "ARG7" : "X8",
}

# register the Primitive Set
PSET = gp.PrimitiveSet("MAIN", arity=len(uvs))

# rename ARGS to match the dataset
for arg, des in uvs.items():
    PSET.renameArguments(arg=des)



# adding to pset

operators = (
    (operator.add, 2),
    (operator.sub, 2),
    (operator.mul, 2),
    (math.sin, 1),
    (math.cos, 1),
    (operator.neg, 1),
    (pdiv, 2),
    (plog, 1),
    (psqrt, 1),
    #(ppow, 2)
)

for (func, arity) in operators:
    PSET.addPrimitive(func, arity)

PSET.addEphemeralConstant("rand1", lambda: randint(-1,1))


# min fitness object
# objective: minimize mse/mae for y1^/y2^
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# individuals program
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)



# TODO: research optimal configuration from literature
toolbox_master = base.Toolbox()
toolbox_master.register("expr", gp.genHalfAndHalf, pset=PSET, min_=1, max_=2)
toolbox_master.register("individual", tools.initIterate, creator.Individual, toolbox_master.expr)
toolbox_master.register("population", tools.initRepeat, list, toolbox_master.individual)
toolbox_master.register("compile", gp.compile, pset=PSET)

toolbox_master.register("mate", gp.cxOnePoint)
toolbox_master.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox_master.register("mutate", gp.mutUniform, expr=toolbox_master.expr_mut, pset=PSET)

# Fitness Functions

In [10]:
def evaluate_case(func:Callable, case:pd.core.series.Series, target_var:str, err_metric:str) -> float:
    """
    Evaluates an individual, compiled program for a single fitness case (=pd.Series), computes and returns error for prediction and outcome for target_var and model prediction

    Options:

        target_var:
            "y1" (heating load)
            "y2" (cooling load)

        err_metric:
            "squared" (error)
            "absolute" (error)

    """
    assert (target_var.lower() == "y1") or (target_var.lower() == "y2")

    # compute individual with case variables
    prediction = func(*case[0:8:].values)

    # optimal value:
    if target_var.lower() == "y1":
        value = case.values[8]
    elif target_var.lower() == "y2":
        value = case.values[9]

    # compute and return error as defined by err_metric
    if err_metric.lower() == "squared":
        return ((prediction - value) ** 2)

    elif err_metric.lower() == "absolute":
        return abs(prediction - value)
        
    else:
        print(f'invalid input for err_metric! Must be "squared" or "absolute"', file=stderr)
        raise ValueError


In [11]:
# def test_f(x1,x2,x3,x4,x5,x6,x7,x8):
#     return 42

# for _, fitness_case in training_set_master.iterrows():

#     print(fitness_case.values[8])

#     evaluate_single_case(test_f, fitness_case,"y1", "absolute")

In [12]:
# fitness function for all fitness case:
def evaluate_all_cases (individual:creator.Individual, toolbox: base.Toolbox, df:pd.core.frame.DataFrame, target_var:str, err_metric:str) -> tuple[float]:
    """
    Evaluates an individual program for all fitness cases (=rows of pd.dataframe) inside the dataframe, computes and returns the mean for err_metric of prediction and target_var 
    """
    # Transform the tree expression in a callable function
    compiled_individual = toolbox.compile(expr=individual)
    
    n = len(df)
    error_aggregate = 0.0

    # iterate through all fitness cases and aggregate absolute errors
    for _, fitness_case in df.iterrows():
        error_aggregate += evaluate_case(func=compiled_individual, case=fitness_case, target_var=target_var, err_metric=err_metric)
    
    # compute and return MAE
    mean_error = error_aggregate / n
    return (mean_error,)


In [13]:
#TODO: test fitness functions

## Statistics

In [14]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
MSTATS = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
MSTATS.register("avg", np.mean)
MSTATS.register("std", np.std)
MSTATS.register("min", np.min)
MSTATS.register("max", np.max)

In [15]:
def run_tournament(
    pset:gp.PrimitiveSet,
    toolbox: base.Toolbox,
    training_set:pd.DataFrame,
    testing_set:pd.DataFrame,
    mstats: tools.MultiStatistics,
    target_var:str,
    err_metric:str
    ):

    ofstream = open(f"./data/tournament_results.log", mode="a")
    ofstream.write(f"Target={target_var}\nError={err_metric}\nTime={datetime.now().strftime('%m/%d/%Y,%H:%M:%S')}\n")

    toolbox.register("evaluate", evaluate_all_cases, toolbox=toolbox, df=training_set, target_var=target_var, err_metric=err_metric)

    # set selection method
    toolbox.register("select", tools.selTournament, tournsize=3)    # TODO: adjust parameter
    # toolbox.register("mate", gp.cxOnePoint)
    # toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    # toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

    # decoration
    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

    # training phase
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, NUM_GENERATIONS, stats=mstats, halloffame=hof, verbose=True)
    except:
        print("ERROR: Evolution throw an exception!", file=ofstream)
        ofstream.close()
        return

    ofstream.write(log.stream)

    for elite in hof:
        winner = elite
        # print (elite)
        # plot_exprTree(elite, "Best Solution")

    # testing phase
    winner_func = gp.compile(winner, pset)

    err_agg = 0.0
    n = len(testing_set)

    for _, case in testing_set.iterrows():

        if err_metric.lower() == "squared":
            err_agg += (winner_func(*case[0:8:].values) - case[8:9:].values[0]) ** 2
        
        elif err_metric.lower() == "absolute":
            err_agg += abs(winner_func(*case[0:8:].values) - case[8:9:].values[0])

    mean_err = err_agg / n

    ofstream.write(f"Mean {err_metric} error computed for testing Dataset = {mean_err}")
    ofstream.close()

    return log

In [16]:
# target = y1
log = run_tournament(
    pset=PSET,
    toolbox=deepcopy(toolbox_master),
    training_set=TRAINING_SET,
    testing_set=TESTING_SET,
    mstats=MSTATS,
    target_var="y1",
    err_metric="squared")

# target = y2

   	      	                              fitness                              	                      size                     
   	      	-------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg       	gen	max        	min    	nevals	std        	avg 	gen	max	min	nevals	std    
0  	300   	2.3284e+08	0  	4.64839e+10	112.834	300   	2.83691e+09	3.19	0  	7  	2  	300   	1.30406
1  	145   	19772.1   	1  	4.03281e+06	112.834	145   	237186     	3.16	1  	10 	1  	145   	1.4791 
2  	158   	161099    	2  	1.76712e+07	88.3417	158   	1.46917e+06	3.15	2  	10 	1  	158   	1.57295
3  	179   	16647.8   	3  	475265     	112.834	179   	72044.1    	3.03	3  	11 	1  	179   	1.63985
4  	189   	7525.51   	4  	424300     	112.834	189   	44635.4    	2.76667	4  	13 	1  	189   	1.57233
5  	199   	1833.36   	5  	92358.3    	112.834	199   	11815.9    	2.34333	5  	8  	1  	199   	1.05457


In [17]:
for line in log:
    print(line)

{'gen': 0, 'nevals': 300}
{'gen': 1, 'nevals': 145}
{'gen': 2, 'nevals': 158}
{'gen': 3, 'nevals': 179}
{'gen': 4, 'nevals': 189}
{'gen': 5, 'nevals': 199}
